In [1]:
# Chat with an intelligent assistant in your terminal
from openai import OpenAI

# Point to the local server
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

history = [
    {"role": "system", "content": \
"""As a creative assistant, I'm here to help you craft compelling, SEO-optimized YouTube video descriptions that stand out. I will not help with other tasks. Here's how we can start:

1. Information Gathering: I'll ask a few questions to understand your video better, including its title, content, target audience, and any specific keywords or calls to action you want to include.

2. Content Creation: Based on your answers, I'll draft a description that captures the essence of your video, engages your audience, and enhances discoverability.

3. Review and Refine: We'll review the description together, making sure it aligns with your goals and preferences, including the use of hashtags or social media links if desired.

My approach ensures that your video description is not only clear and engaging but also tailored to meet your specific needs and audience expectations. Let’s make your content shine!
"""},
]


In [2]:
def get_completion(input):
    message = {"role": "user", "content": input}
    _history = history+[message]
    completion = client.chat.completions.create(
        model="lmstudio-community/Meta-Llama-3-8B-Instruct-GGUF",
        messages=_history,
        temperature=0.7,
        stream=True,
    )
    return completion

In [3]:
def wrapper_chat_history(chat_history, history):
    chat_history = history[1:]
    return chat_history

In [4]:
def converse(message, chat_history):
    response = get_completion(input=message)
    user_msg = {"role": "user", "content": message}
    history.append(user_msg)
    ai_msg = {"role": "assistant", "content": ""}
    history.append(ai_msg)
    partial_message = ""
    for chunk in response:
        if chunk.choices[0].delta.content is not None:
            _tkn = chunk.choices[0].delta.content
            partial_message = partial_message + _tkn
            history[-1]["content"] += _tkn
            chat_history = wrapper_chat_history(chat_history, history)
            yield partial_message
    

In [6]:
import gradio as gr

history = [
    {"role": "system", "content": \
"""As a creative assistant, I'm here to help you craft compelling, SEO-optimized YouTube video descriptions that stand out. I will not help with other tasks. Here's how we can start:

1. Information Gathering: I'll ask a few questions to understand your video better, including its title, content, target audience, and any specific keywords or calls to action you want to include.

2. Content Creation: Based on your answers, I'll draft a description that captures the essence of your video, engages your audience, and enhances discoverability.

3. Review and Refine: We'll review the description together, making sure it aligns with your goals and preferences, including the use of hashtags or social media links if desired.

My approach ensures that your video description is not only clear and engaging but also tailored to meet your specific needs and audience expectations. Let’s make your content shine!
"""},
]

demo = gr.ChatInterface(fn=converse)

demo.launch(share=True)


Running on local URL:  http://127.0.0.1:7863
Running on public URL: https://4c65229df98593a864.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
